In [ ]:
input_test = """##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^"""
output_test_p1 = 10092
output_test_p2 = 9021

input_simple = """########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

<^^>>>vv<v>>v<<"""

input_simple_p2 = """#######
#...#.#
#.....#
#..OO@#
#..O..#
#.....#
#######

<vv<<^^<<^^"""

In [ ]:
import numpy as np

char_map = {"#" : 9, "O" : 1, "." : 0, "@" : 5, "^": "u", "v" : "d", "<" : "l", ">" : "r", "[" : 2, "]" : 3}
direction_npa_map = {"u" : np.array((-1,0)), "d" : np.array((1,0)), "l" : np.array((0,-1)), "r" : np.array((0,1)) }

def parse_input(inp, p2 = False):
    grid, actions = inp.split('\n\n')
    if p2: #widen
        for f,t in zip(["#",".","O","@"],["##","..","[]","@."]):
            grid = grid.replace(f,t)
    grid = np.array([[char_map[char] for char in line] for line in grid.split('\n')])
    pos = np.argwhere(grid == 5)[0]
    actions = [char_map[char] for char in actions.replace('\n', '')]
    return grid, pos, actions

def find_first_open_and_wall_from_pos(g, pos, d):
    to_find_list = [0, 9]
    
    if d == "u":
        l = g[:pos[0], pos[1]][::-1]
        return [None if np.all(l != to_find) else (pos[0] - 1 - np.argmax(l == to_find), pos[1]) for to_find in to_find_list]
    if d == "d":
        l = g[pos[0]+1:, pos[1]]
        return [None if np.all(l != to_find) else (pos[0] + 1 + np.argmax(l == to_find), pos[1]) for to_find in to_find_list]
    if d == "l":
        l = g[pos[0], :pos[1]][::-1]
        return [None if np.all(l != to_find) else (pos[0], pos[1] - 1 - np.argmax(l == to_find)) for to_find in to_find_list]
    if d == "r":
        l = g[pos[0], pos[1]+1:]
        return [None if np.all(l != to_find) else (pos[0], pos[1] + 1 + np.argmax(l == to_find)) for to_find in to_find_list]
    raise ValueError

def update(g, p, d):
    p_open, p_wall = find_first_open_and_wall_from_pos(g, p, d)
    #No open positions, push impossible
    if p_open == None: return g, p
    #If either no wall or first open spot closer to position than first wall
    if p_wall == None or np.any(np.logical_and(p_wall > p_open, p_open > p)) or np.any(np.logical_and(p_wall < p_open, p_open < p)):
        P = p + direction_npa_map[d]
        g[tuple(p_open)] = 1    #box goes into first open position (is overwritten later if no box was pushed) 
        g[tuple(p)] = 0         #current position is now open
        g[tuple(P)] = 5         #position moved into is now robot
        p = P                   #update robot position
    return g, p

def solve(inp):
    grid, pos, directions = parse_input(inp)
    for d in directions: grid,pos = update(grid,pos,d)
    return sum([x+100*y for y,x in np.ndindex(grid.shape) if grid[y,x] == 1])

def get_push_positions(grid, pos, d):
    set_prev = set()
    set_prev.add(tuple(pos))
    list_push_positions = list(set_prev)

    while set_prev: #starts out with only current robot position, is updated with everthing it "bumps", then everything which is 2nd order bumped, so on
        set_new_pushed_into = set()
        for P in set_prev: set_new_pushed_into.add(tuple(direction_npa_map[d] + P))
        set_new_boxes = set()
        for P in set_new_pushed_into:
            if grid[tuple(P)] == 9: return False #Bumping into walls means entire push fails
            elif grid[tuple(P)] == 3:
                set_new_boxes.add(P)
                if d in "ud": set_new_boxes.add(tuple(direction_npa_map["l"] + P)) #Add 2nd half of box
            elif grid[tuple(P)] == 2:
                set_new_boxes.add(P)
                if d in "ud": set_new_boxes.add(tuple(direction_npa_map["r"] + P)) #Add 2nd half of box
        list_push_positions += list(set_new_boxes) #Remember all positions that will be have to be pushed to update grid
        set_prev = set_new_boxes
    return [np.array(p) for p in list_push_positions]

def grid_print(grid):
    for line in grid:
        for char in line:
            print(char, end='')
        print()
    print('\n')

def solve_p2(inp):
    grid, pos, directions = parse_input(inp, p2=True)

    for d in directions:
        list_push_positions = get_push_positions(grid, pos, d)
        if list_push_positions != False: #If push didn't find a blockage 
            grid_next = grid.copy()
            for P in list_push_positions[::-1]: #This ordering avoids erronous overwrites
                grid_next[tuple(direction_npa_map[d] + P)] = grid[tuple(P)]
                grid_next[tuple(P)] = 0
            pos += direction_npa_map[d] 
            grid = grid_next

    return sum([x+100*y for y,x in np.ndindex(grid.shape) if grid[y,x] == 2])

In [ ]:
assert solve(input_test) == output_test_p1
solve(input_full)

In [ ]:
assert solve_p2(input_test) == output_test_p2
solve_p2(input_full)

In [ ]:
input_full = """##################################################
#...O.....O.O.O#..O.....#.OOO..O#.....O..O#..O#..#
#.O.#.O#.........#O..O..#O...O..O#......O..O.....#
#...O..O...OO..O#..OO#.O.......................O.#
##....#OO....O.O#.OO#.OOOOO...#OO.OO....#O....#..#
#O...#.O...###.OOO...O.OO..#....OOOO#O.O.......O.#
#OOOO..O#O.O.O..OOO..O....O......#OO.O..O..O.....#
#....#O.O....OO.......O...OO..O.O.O..O......OO#O##
#....OO........#O...#.O....O.OO...O.OOO.........##
#.#OOOO..O.O......O..O...OOO....OO.#OO.#.........#
##OO......O.OO..#...OO.O##.O....OO..O.....#......#
#O..O.##O....O...##.O....O..#..O...O.O........O..#
#O......#.....O.OO..O......O...#...O....O.O..OO..#
#...O..OO.OO.....O#OO.....O..OO.#.....OO..O..##..#
#.#.O#.O..O...............O..O......#....#....O..#
#O...O....O.....O....O...#..##.#......O..O...##..#
#..O.OO.....O.##OO.OOO..O......O.O.O...O..O.O.O..#
#...##..O..O.O.........OO.O.#.O.O...O...OO......O#
##...O#.......OO.O....##....##O.OO.OO.O....#.....#
#.O.OO#...OOOO#..O..O..#..OO..O......O..O....#O.O#
#..O........OOOO#OO.OO...........O.O..OO....OO...#
#..OO.OO#.OO...O........#..O..O..#OO#.....O..O...#
#O....O..OOOOO...O.#...O.#.#.O.........OOO#.O...O#
#O....OOO.O...O.OO......O.O#.O.O..#O..O#.OOO..O.O#
#O........O.OO...#...O..@......O.O..#O..OO.O.#..O#
#OO....#.O......OOO..O.......OO....O#.OOOO.O.....#
#.O.O.O...OOO..O.OOO.O#.O#O...OOO#..#.......OO.#.#
#.O..#.OO..#.#...O.........O....O..O.....O.OO.O.O#
#.O#OO.O..O#....O......O...O#.O..O.OO.O...#O....##
#..O.OO..O.#..........#.O....OOOOO#OO.....OOOO.#.#
#.O..O.........O##O....O.#.#......OO.....O..#.O#.#
#.#..O#.#...#...O.....O..#.....OOO..O#.O..O##O...#
#.O#..O...OO..O...O.#..O.O....O..OO..O.......O..O#
#...O.OO...O....O.......#O.O.O##.O...O..O#...OO.O#
#....O.OO...O....O...O....#.#..#..O...#.O.O.#....#
#.OO.#.#..OO#....O.O..#..#..O.O..O......O..O.O...#
#..O...O#.O...OO....OO...O.#..O...#.....#...O#O.##
##..O..O.#.O...O..O..O..#.......O..O..O#..O.O...O#
##....O...O.O..OO......O...O.OO..#.O#..........OO#
#O...O..O#O...OO...OO.O.....O...O.#....O.O..O..O.#
#....O...O.OO....#O.O...#.#......O.#...O...#...#.#
#.....O..O.O...OOO#O..#OO.O...O...O..OOO...O...O.#
#O#O....O#....O.#..OOO....OO...#...#......#..O.O.#
#OO..#.....#.O...OO...O#.O.O.O..#....#O.O.....#..#
#...O....#O.O.O..#..OO...#.OO....#..#.O......O...#
#O....OO..OO.OOOO...OO....O.....O...#..OO.#.#....#
#O....O..........OOO.O..OO.O..O..#..O#..OO..O...O#
#.O.OO..O....O....#O#O......#...O###...#O....O..O#
#.#O.....O..O...#O.....#O........O.OO....O.#....##
##################################################

<<>^><>v^v<<<v<>>>vv>v><>>v^>><>v>>><><<^^<v^^>v^><<^><<v<>^v>v>><v>>^v<<<v><<<<vv>^>>v^v>>v>>^v>v>vv><>^<><<>v>v>^>>^<^v<^>v<<><^>^^>^<^><<^><v^><^><v>v><<v><>^<<v<<<<<^^<>>>>>>v^^^>>^v<v>>><^>v<^<^v^>><vvvv>^><vv<v<>vv<^v^v<vvvv^>>v<^^vv^>vv^<>>^<v<><^v^<>^vvvv>>v>^<<v>vv><<<>v^v>^<<<v<v>>vvv^>vv>><<>v>v<<<<^v<>v>vv><>^<vv<<<<^<^>^v^v^v>v^>>v^v<<v^^>vv>^>><<><<>>^><>v>^>^^^>^v>>^^v^<v><>vv>v<>v><^<^v<<vv>>^^^<v^^^^^<<^v<^vv^>v><<^^^<^v<^v^v<v>vv^^>v<><>v^<<v>^><vv>vv<<<^^vvv<v>vvv^^^>>^v<<vv<><^<^<vv>^v<vv^v<<^^^<v>v^^v><><><^<^><v>v>><>^^>^<<v^<^v<v><<v><>^<<^<>^>v>^><<v^>>v^^><v><^>v<^<v<^>v<<^<<^>^>v^><vv>vv^v>><>^^^<>>^<^^vv<<<<^>^>^^^>v<vv<^v>>^v^^<^>^<^<<>v^>^>v>>>v<v<^v>v><<^<^vv>^><v<vv>^^^<<<v>vv><><^^<>v>><>^><^>^v^><^vv>><vv^v>^<^vv^<^<v><<v>v^>^^^<<<><>vv><v^<><^>^<^<<<>^v><vv>^>v^v<><vv>v>^><>>^><><^^v<<<<vv<>^vv<^^v^><>vv^^^<>>vv>>><<><v>>^>v>>><>v^<^<^^><v><vvv^>v<^v<>v^><^>^^^<^^v<^>^<><>vv>v<^v<v<>^^^v^>>vvvv>><>^>vv>v<v<v^v^>>v<>^v><v^^v<<v^v>><^>>>vv<<>>^vv^v>vv^<>
>^<<v>>v^<>^^<<^<^>>v^>v^^v>>><>^<<^><v>>^>vv<>v<>^<>^v<v><<<<<v^^>^>v<v^^^<<<^<^<<<<>>v<^v^v<<><v^>>^>^^<^<<^^^v^>>^v^>><>^<^<><v^>^^^vv<^^v<>v>v>v<^<v<v>^v^^><>>>^><^v<vv<>v<><vvv<<^<<vv<^>^<v<<>>vv^<>>v><^<>v^<<v>^<>><>><<>>^<^^<>^v<^^>>>^><v<vv>>^vv<v^>vv^>^^<^v>>^^>^>^^^vv^^>><<v^<>>^^<vv^><v<vv<><<v>>^^^v^<<^>v>^<^vvv>v^^^>><<^v^<^v>v^vv^^vv<v^><vv^>^><><^>v^><<<><^v<vv<>^vv<v><>^^><<<^><<>><>>^v>v><<><^<<<v<>^v>>^<<^v<>><^v>^>^<vv<^^v><v>v><<^vvv<>>v<^^>^>>><v<><<>vvv^^v^<v<v<>vvv^^<>>>vv><^>>vv<>v>v>>>^^<<<>^v^^^^^vv><v>^v^v^><v>^vv>vvvv^^^>><^^v<^><^><vv><<v^><<>>v>>v><^>^>v^>v<^>v^^>^>>>vv^><>v<<><<^v<^<>v>^^<<^<<^^^^^^<><>><^vv>vv><>^<^>v>^vvv^<v>v^>^vv<^>>^^^^<v<^^<>>^vv<<<^v<vv>v^^<><><<<><v<v<v^v>vv>vv>^^>><<>>>>>><<v<<<<vv>v<v^^v^>v^<>v<>v^>^^v<^v>>^>^<^>>>^<<<<v<vv^^>>^v<>^><<><^<>v^^^<><<^<<>>vvv^v>>>><<v<>^>>^<>>>vv^<v^v<^>^v>vv><<v<v<<<<v>^vvvv^v^<v>vv>>vv<^v<<v^<vv<<><v>^><v<^>^<^>><<><><<>v>><><>^^^<><^v<v<>>^v<<^v^v>vv<^^vvv<>v><v<^<<>^v^v>>v<<>^^<vvv<^^<><>>>^v>v
>^v^v^<^v>><<<^vv<>><>^<>^<<<^<^^v<<v<v<v<>>><^<vv<>^v<><<<>>^<>v^^><<<^<<<^>vvv<^<<<<<<<><>^<v>v^^vv<><>^>>v^v>^<vv^vvv>v^^<^^^<v^^<^^<<vv>^^v<<<<<vvv>v<>^v^<<^^><>v<<<^v^>vv><>v>^v>v^^v^<<<>v<vv><>>><>>><^<v>v>>v>^>v^<><v^<>^^vv^^^<>^v>^v<v^v><<v<<v<>>v<v^v<v<<v^v><<^>>^^v>>^>^>><v>v^^^v>><><vv>>v^>>>v<>^><<><>>>v^<<^><>><v><^><v<^>vvv^vv<<>v<>v><>>vv<v>^v<<^v^>v<v^<<<^v<v>vv>>^v>^>^<><<v^>v^vv>^vv^^v^><<>>><>v^>><v^^<^>v^^v>v^v<<>v>v>><^^^v<^v^>^<vv^>>^><<>^vv^^v<^>><^v<^<<<><vv^<><v>^<v^v<<>>>>>v<<v<<^<>>^<v^^<vv^><><v>>vv<^^><>v<^vvv<<<^^^>^v><v<v>^^<v<v<vvvv>vv>><>^<>>>^><v>^<<v<v<<<^<<^>v<>>>^vvv<v^v<>^^v>vvv<^v>v^^v>v>vvv><<^<vv><v^>vv^v>><>v^v><^<^v^v<^v^<><<<v<^^><v>^><^<vv^v<<><^>^^^>vv>^vvv><<>><^<>>>^<^><^>^vv<><^>><><><>^><>^v>>v^^>vvv>v<><<v<<<>v^<vv>><v>><>>>v>^>^^^v><^<^v<^<^<^v^<v^^<<<v<<v<^>v>^v>><v^<v^<v<<v<<>^>^<vv>v>v^v<<^v^><^>>>^^^<>><^^><^v><><v^vvv>v^v>>v<^^><^vv><<^v>^v^^<^vv>v<v<<vv><>^<v<<v<<<^^^v<<>><^<<>>>>^^^^^<v>v<<v>><<><<>><v^v^>v>>>^v>>>^^^<^^>>^><^<
v<vv<^>v<v<><v^<>^^<vv><<<^<<<>^^>>><^^<v^v<^<>v<>v<^^^v<^^v<<v<v^<><^^<>v>v<>v>v^^><<<^^v^<^vv<>>v^>^<v<>vvv>><v>^>v<v^<^><<^>v^^^^<>^>>>^^><<<v^^>^>v>v>^<vv>v^v>^<>^^<v<<^v>^<<<>v<<>v^<<><<>vv^v>>>>v<>^v><>^v>v<^<>v^<^>v<><>^<^vv>^><>^v<^>><vv^<^<v>^v^<>>vv^><<^^><^<^v^>><v>^>v<v^v^>>>>>><^><v^<v<<<^<^<^v><<<<>v^<^>^<v>^^^<^<<v>vvv^^^v><<v>><<><^^>^v>v^<vvvvv>>>^>v>>>^<><^v^v^<^^^v^<vv^>vv><<v<^v^<v><^^<^<^v>>v<v^<vvv^>^>v>v<<^>^^^>^<^><>^v<^><>v<>>v^v<^vv^>v^vv^<<^<<<<^>^>>>^vvv>>vv>^>>v>^vv><<>vvv^^<^^>^<><^v><<^^v^<^<^v<^vv<><v>>>^>^><>v^>vv^^>^vvv>>>v>>vv<<<<v<^>^^^>^><>^^>v^<<^^>v^vv>v>^>>^^^^vv<^vv<<>><^v<vvv><<>v^>^^<<vv<<<<<^v<<>>^v^<>v^<<v><>v^v^v<>v^^^<v^vvvv^<<>^vv<><<>^>>>^>^^>^^^^^v>^^^^^^v>^v<v>^v<v>v^^>^<><^<v<>^>><>^<<<v<v>vv><^><^<<><v>>^^v^vvv<>>^>v>^^v<^v<v><^v<>^vv<<^v><v^v^v<<^^^>v>vv<<>>>>v<>vv<^<v^^v^<vv>^^>v>^>^^>vvv>^<v><^><<v^v<v>>^^^<^v<<<^vv>^<v<>><><^<<^^vv>>^<<^^v^<<^>^<vv<^v<^>>>v<^^>v^^>v^>>^vv^^>v<<v^>^<><<<v<^^><^vv>><v^^<^^<<vvvvv^>^><<^>^<<>>>^vv><
^<^<^<^^vv><^><<<v^^v<<>>>^v><v>><<<<<>>v^>^^v<^^>^<<v<>^v<vv^^<vv^v>>^><<v<>vvv>v>v<<vvv<>^<>^><^v<>^^^><v<<^<^<v>^^>^^v<^>vv><^^<>v^>v>^^>v^>v>>v>^>vv>v^<<<<v<<v^<>v>v<>>v<^^<<^>^v<<>v>><>v^v<>^^<<v^<v<vv^><^>vv^^>v<v^<^><<^vv<>v<>v<^^><^^v^^<^>^v^>>v^v^<<vvv^^<vv>v>v^>^<>^<<^v><>^v<vvv^<v><v<<^v<^v<^<><><v<^^^^v>^^>v^^^<>><^<>^^>^<<<^^<^>^>^v>>>v^vv><<<^<^v>vv<^<v><^^><v<>v<<v<<>v>>>>>^v>v<>^vvvvv><^^^>v^>>v^^<^^>^<>^^^^<>^<>^<v^v<<^>>v>v^v^^<v<<v^^^<v>v<>^^><v>>^vv^<<^>>>^^vv>^<>vvv<<>>^><>><<v^^<^<>v>vv^<^<><<v^^v<<<>v<<<vv^^^<>>^<>v>v>>v^>v>><>vv>>^^^<v><>^^^v^<<><^vv^><>^>v<^><^>>v^^<>v^v^v^^v>vv>>^^v^^v^v<<>^<><^^v>v><<<<>v^^v>^^<><^^^vvv^<v^vvv><>v>>^^^v^^>^><v<><^><>>>^<v^^v<^<<<vv<<v^v<<vvv^>^<v^>>>^<>vv>><vv^v>v<>vvv><>>vvvv>v<<<^^>v<v>^^>>>><v^v<>>v<v^v><^<v>vv^><<^vv>><<vv<>^^<v<>^v><>vvv^v<^^><^^v<>^<<^^^<v<>v<^>v^^<><<v<>>v^>>>^<^^>vv^v^<>v<>>vvv^>^<v><>v^^v^<v<>v^>>v^v<<^>><><^>^<>^v^^vv<><v>><^^v^v^<^>>^><^v<^>vv^<>v>^>v<<>v<<^>^<<<>v><<<^^>^><v><v^<<><<<v^>>v^><<<v<>
v<<<<><<>v^<>v^^><><>v<vv>^v<vvvvvv^^>v^v><>^^<<v<vvv>vvv^^v<^>vv^<v<vv<<>vvv<<v<^^><vvv>^^<^<>>^^><<<>><^<vv<><<vvvv^^^<<<<<^>vv<<^>v^>^v<^>^<>>^v^v>^<v^v>>v>>v<v><<>^>^<v<<><^vv>v>^><^^>><^^vvv<><>vvv^<v<v>^>^^<vv<<^vv<^^><>^v^<^<v><^^>v>^v>v^>v^><v<>>^v><v>vv<^>>v^^v^^<>^v>v>^<<><^<>^<^^>>^^>v^v^>>v>^>v^>v<^<>>>vvv>><v<^v<^vvv>>>^v^>>^vv<v<<v><^<>>>^>vvv>><^vv<<<>^>^v<<^^<^^^<>v<v>^><^<^<^><><v<v^v<>>^>^<<<<v<^^<>>>vv<<^<^v>^<<>>>vv^v^^><<vv<<vv<<^^><<>>>><vv<v><v>><^<v>^<<vv<v^<>v><v<>>v<<v^>^<<>^<vv^>>v><v^v>^^^v>>^>>>v^vv>>v<vv<v<>>^>>><^<>>v>v><<v<v<>v^>v>>vv<<^>v^><^v>v^>>>><>^>vv^^^^v<^^<<^>^^>^><><v<<<>v^<v<^v^>^>^^vv<<<><v^v^v<<^vv<<^><<vv>><vv>^vv^<v>^^v<>>^vv>^>v^v>>>>vv>><^v^^^vv^^v<>><>^v>^<v<>v>v^v<<^><>v<<>^><^vv>>>>^<><^<<v<vvv<<>^><v>^vv^^<>v>v<^><><^vvv<^^<>v^>^<^^v^v><<^>^v^v<v><<v>^<>>><>vvv^^v<^>>vv<v<>^^v>^>v^^>><>>>^>><<><<>><^<^v<v><>^^v<><v><<^vvv^<>vv^>><>^<^^v^v>v^>v<<>v<<^^>>v<<v<^v><<^^<^>>>^<>v<><>^<<v^v>vv>^<<^v^><><v^>v<^>><^><>vvv<>vv^^^<><v>><<v<>^<v
vvv>>v^^v>v^^<><v^vvvv<>v^^v<><v^>^v>v^^v^>v^>^<>>><>>>vv>v<v^v>^vvv<<>^>>v^vv<vvvv<^v<^<<>><v^v<><<<^v>^>v^<>^^v><<v<><vv>>v<vvvv>vvvv^^>^><v<v<^^vv<v^^>^><vv<>vv<>^v^^v^<<<^^^^<><vv<<^^<^>>v<^vv>><v<>^v>v^v^^>^v<><^v><>>^^v<><>^>><^^>^>v^><v<^>>^v<>>>vvv^^^<^v^>>^<>^vvv^>>^<^^<><^v^>>>v<<^>^>^^v<vvv<>v^v^^v>v^^<<<>><<^vv^^<>><>^^^v>^v^vvv^>><v<^^>^vv<^vv^<<<v>>vv^<^v^<^^><<v^^^<^<>>>^<^v><>v<^<<>vv<>^<v<^>vv>v>^>^^v<<<^>^v>^<><v<^^^>>>>^v>><^^>^<><^<><<vv^>>^^^^>^>^v<<<^v<v^><>^>>vv^v<>v<^v><>^v^v<^><>>>^v><>>^vv^^>v^<vv^^<>^<^>^<>>v>^>^<v>>>vv>><^>>^v>>>^<vv>v>^<^v>><v^<v<v^>v^^<^>>><^v>>v>v>^<^^<<>v^v<<v^v<v^vv^<^^>>>^v<>>vv><v<vvv<>v^><>>^>vv>^>>^v^>vvv<^^<v<><v>>^^<<<^<^>v^><>>^>^v^vvv<<^>^<v><>>>>><v^vv><>>^<v^>vvv><<v>v^<>vvv>>><<^^^><<^vv^v><<><vvv^<><^^>^<>vv<<v^>^>>^<^^<<<>><^><^<^<>>v>vvv<><><^>^><v^v>v>^v<>>^<v<vvv^^v>>>><>vvv^^^vvv<>v>>>>v>>^>>^v^v<<<^^vv^^v<>^<>>v<v<<>^>v<>><>^<^^^<v>^><^vv>v^^^><v^<vvvv^<v<v<^><vvvv>>>>v<vvv^<vvv<>v>^^<>v<<<v><<^^^<<^^v>^>^<^>^<><^^<>>v
^<v<v<<^><>>vv>^<<>>>^^>v>v^^>vv^>><^<>^vv<>v<v^><vv<>>vv<v>^^<^><>^<v>v>vv^>v^^>>v>>^^<^<<><v<>v><>><^>><>v^^<v>^v^>^<v^^<>v>>^vv>vvvv<<<^<v<^<>>^>>><^><<<v>v><>^>vv<^v^v^^v><<^vv^<v^>>><>^^<^<^<vv<<<vv><<>v<^<>>^vv><^><vv^>^^^>^v>vv^v>v<<><^^<^<><^<>v><<>><v^><v<<<<<<vv^vvv^^v<<<>v<^>><>^vv>>>><vv^v>^v<^vv>>>><><^<<^v<^<^><<>vv<>^vv^vvv<^>v^v^<v>>^v>>>>v^^vv<<>>^^<^<>>vvv<>v^v<<v><<v^<<>vvv^><vvv><>^v^v^>vv>v><^vv^^v<<v<^^>>^<>v^>v<v>^^v>^><v>^>v>>><>^><vv<v>^v>vv<>v<^^>^v<v>^>^><v^^^^<<vv>>v>^^^>><^v<><v>^<<<v^^>v>>><v>v^^<^<<<vv>>^^v<><<v<<<><^^^<vvv^v>>^^<<^^v<><^v>^>>^<<vv<>^<<^^v^vv^^><^><<>v^^<^v><><^^><><^><^>vv>^^vv^><<^<>^^vv^<v^^v^><>^v^v<>v^<^v^>^<>v<^^<^<^<^vv^^<<>^^^<<<>vv><>v>^v^v^<v^^v<^>^><>v<v>>^<<^^^<v>>^<v^>>^vv^<v<<><^vv^<v^vv^v>^^<><^<>><^v<v>>>>vv<>vv<>v^>v>>^^vvv<^<^><^v<>v^v<^>>v^<<>><^>>v^<^^<<<>vvv^^v^^>>><v<v^^<v^v<<vvv^<>>vv<v>v>^>v^<vv>v<^<>v<><v<v<>v><^>v>^<^>>><<<<vv>v<vvv<<>>^^>v>><v<vvv^^<v^>^v>^v^>v<v^>^v<v^><><^>>^<^><^<>^>^<^^^^<<<^vvv><>v>>^<>>^^^
^<v<^^<><v>>^>>v<^<<^>>v<<v^>>^<^>v^v>>v<>vv<^vv^v<<v^^><<>v><^^<vv><>><<><v<><v>>^v^><v^>^>>^vv^v^^>^>v^>>v<<>^vvv<>v^<<^<<^>^<vv^<<<v>^<>>><^<^>v>v><v<<<v>>>>>v^>^<><<>v^^vv<<<>v^v^><<^>vv<>>^><^^<^>^^><<<>v^<v^>^<v^v<<v^<>>^^^><><^<<>^v<vvvv>^v^>^>>>^v>>^v>^^^<v^vv^^<^>v<v<v>>^><<<^>^<^<<v>^^^v^<^<^<v><<>><<<<v^<^v>^vv^<>^^<><>v>>v>^>vv<^><vv^<^v<<<<v><>^v>vv^>>v<<v^>v^<vvv^><<^v>vv<^v^<>v^v^^^v^<<v<><v>vv^v^>^<vv<<vv><^v^^v^^><v><^vv^>>><>>v^^^<v^^^^<>^<^<>v><^>^v^^v^v<^vv^>^<<^v^v^^v<^v^<^^><>v><v><^v^>v>^><vv<^v><v<^v>^^<^v<^<v^<vv^vv<<<<^<v<>>^>v>vv>v<^^>^v<v>><<v^<><^^>v<>^^>^<<<<vv>v><^><<<vv><<^v<><<^v^^>>^<v>^<<v<<<>^v<^>v^^<^<^<<>^v<><v<vv^^^v<^^>^v<vv^v>^<vv^<>^<<v>>vvv<><><v<<>v^vv<<<<>v^^v><^vv<^>^>v<>vv<vv^><^v^>v><v<>>v<^vvv^<vv<v>v>>v>><>>v^>vv><<>>vv^^v>v^v^v<>><>>^vv>v^vv^>^>vv><^^v^^>>>^v<^<<v>v<>^^<v<v><>>^^<>><<<<vv<vv^^><vv<^>v^<^>^v<<v<>^<v>^^v^v<^^v^<><vv^^<v^^>^^^>v^<v<v>v^<v^^<>v><>^<v^<v^v<v>v><<^>v^^vv><>>>v<v^v<><<<v^^<v>v<^^<<<<v>^vv<>><v^v>>^>v<vv>>^<>^
^v^<>><^v<<>>^^<<^<>>>v<<^<>v><^^v><^v>v^v^v<>>v>>v>^v^v>^>>><<<^^>^>v>>^><<^>^^<>vv<<v<v^vv>>>>>^<<><<vv><v^^><v^v><<^^>^^<^v<^<<v>^<<v^>v^>>^<v^>^<>^^v<><<^>>v<^v<vv<>v<<>^vv<v^<vv>>>vv<^<>><v^>v>v^v<v^^v^>v>>>>^>>^v^v<>>vv<^^>^^^^<v>^^vv^vv<^<vvv>>v<^><^>v>>>>><<>v<v>>><>^>>><>v>>>>>^<^>^<^<>^vv<<^<<<<>^<<<^<^^<>>>^<<^><^<<<v>^^^^v><><<<>^<<<v><^>>v<><v<<v<vv<^v^v>v>^^<v>v><^<v><^>v><<<><v^><<^v^>><vvv^<>v<v><<<v^^^vvvvv>^<vv>^<><^v<<<>v<>v>vv^v>^v<^>vv^><>^v<^v<<v^v^vv^v^vv<^v>>vv^<v>>vv<<^v^v<^v^<^>>vv^>vv><v<^>^>>^<v<^v>vv<>>v^<^>v><<^<vv>><>^>^>^>v>^^<^v<><v^^v<<<v<>>^^>^>^><vvv><>>>><v<^^>v^>v<v^<><v<v>v^v<^^<^>>^<^<>^^>vv>vvv>^<>>v^v^vv<^^<<vv^^^^><v^>><^<<v<v^vv<><><<^v^v<>>^<^vvvv^>^v<>><<<><^^vv<<<^<<v<v^>v^^>v^v>^<>>>^>v>>^v^^^v<v^><>>>^^v^^v^<>>^>vv<>^><v^<<<><<^^<^<<^v^v^v>^>^<<v><>^<^^v^><<><^v^><vv>>v>^>^>>><>v^>><^v<<<<<^v>^^^^v^v^vv^^>^^>vv^<^<v>>^<>v<^>><>v<v^v<vv^>^^v<^^<v^>v>^<<>^>^<<v<vvv><^^v<<<>^^^v><>vvv^^<^<>^>v><>v>><^<>>^>^vv<><vvv^^v><v<>>^>vv>>v^<^<><vv^v
<<>>v>^^<<<vv>v><<<<v^v<<<<^v><<<vv^vv^^^v^><^vvv>^<<^><>v>><<<^v^^>>^v>v<>>><v^v^v>>^^v>vv^v>v^<^<vv<v>vv<^^^><^v>><>>>^<v^^^v<^^>^<>^^^<>vv^<>><<<v^^v<^v^v>v<^^^<>v^>vv^<<^v<<<<<<>v>>>><<<^<v^<<vv^>vv<^><<>^>>>>v<v^>^>>^v>vv^v^v^<v>v>v>>>v<<>v><><<^^>>>^vvv^^^<v^v^><^v^<<<^^^^^^<<v^<>^v^>v<vv>><vv>v<><>v>^>v<>vv^>^<>v<<<vv<^>^v^<^>^v^<^>^v<<>^vv^><>vv><<<^>>^^^^>^<>>v<<>vv>v><<><>><>^>^>^v^<^><<^v^>>^<v<<<>^<<>><^>v>^<<<<^<^^v^><>>v^^>^<<<<<v^^<><>^v^^v<>>^^^<<<v>v^><v^^<<<^>>v>vv^v^>>>vv>>v^<<^v<<^<><^vv>^>^^vvvv^<v<^<<v<^><^^<^<vv<<<^^vv^^^<<^^<>v<>>^^^v<^<^^<v><<<v^>^v>^<<vv^^^^v^<<^^<^^<<<<<v>^^<^^>>v<>vv><<<vvv<>^^<<><><^^<v^>>^><>v>vv<<><^vv<>v>v<v>^^v>><v><<v<^>><>^><^<<<>>v<><<^v<^<>>>^^v<^vvvv<<>v<<<^<>^^<<^>vv<><>>v<><^vv<^<^<v><<vv<^v<v<>^<<<v><^^<^<><<>>v<^vvv^^<><<^<<>^>>vv<^v>vv^<^^v>>v><^v<>v>v^v<<<<<<<vv^^^<vvv<v^<><<><><^>^>v^>v^^v<v>vvv>^>v<^v<>v<<^^^^<^<>^^^<v^^>v<v^v>v^<^v><>><><^^vvv><^v<^^v^>^vvvv<>^<><<<><>>vv^>>^^^><><<v><^>v^><>^^^^v^<^<<<><<>^v<^v>v<<>>v<v^v
v<^>><^v^>v<v<v^<^^vvv<v>><>v^<>v<^<v^>v<^^<^v>>^^^<><^<^^^^^<<>><><>v^vv^<>v^><v^^^vvv>^v^<v^v>><>>v>>v^<v^v^>^vv^v<v>>v<><><><^<^^v^^>v^>>>^>^<^<^v>vvv<^>><>>>^>>vv>^^vv^>v<<><<^>v<>>><<><v^^<^^v>^<<<<v><v^^<vv^<^>>^<<v^v^vvvvv>>^<^<^>>>>^<><^<>vv<vvv<>>>>>v<^^vv<v^<v^><>^^^>^<v<<^vv<v^<^<v<v>^><<<^<v>vvvv<<^<<<><>v>>>>^vvv><^><^<^>^<v><^<><<v>>^>v<>v<^^<<^^^<<<><^<v^^<<^^><<v>>vv>^^v<>>^>>v>vv<<><<^^>>^<<v>><<>^<v><v<^<^<^<^><^v<^><^><<^>^vv^v^>>v<^><<^>>>^><^>>^^v>^>v<<^^^<^vv^v<<>>^>><v<>>>vvv>vv^v<<^v<^^v<<vvv<>>^<^<vv<^<^<^>>vv<>v^>>v^<>><v<<><vv<<>>>v<^<^><^v<v<<>v<><vvv<^<^<<><>><v^><^^<>><>^vv>v^^<>v^<^^^^^^vv>>^vv^v>vv<<>><<^v^v<v^^^v<<<<<><<<<v<v>>^><^><v<>v>^^vv^^><^v^v>><^>>vv><>>^^v^^<v>^<<^>>v<v>^<^^^<>^v^v<<^v^^^>>v<>v^>^>>v<^v>v^v^^<>^v<<>>v><^^^vvv>v<<<><v^<<^^^^>v<>><^vv^<>><>^vv><>>vv^>v^<v>v<^<^^^><>v<<<>v>>v^>vv^v>>^<<><><v>>>^<<^>v^v<<>^v^<<<<v^<^>><<^v><^v><vv>><v^v>>^vv><>^^^>><<^^v><vv<<^^<^vv>><>v<<<^v<<>>v<<vv<>v><v^^vv>^<^vv><v^<>^<v<v<<<<vvvvv>><vv^v<v<<<
^<^<vv<>^>v^<<<v>vvv^vv^>^^><><>v<v><<^^<v^<><^^^>^^^><v^vv>^v>v>>v^^v><<v^^><>>>>><><>>^>^^>vvv<<><<^v<v^v^v>vv<>^<>>^<vvv<>^><<^v^^<><v>v>>^vv^^<^^>v>>>v<<^v>^<>v>v^vv<>v>>^<v><v<>^^>>^<vv<>^v^^v^>v>v>^<v>>><v<>v^<^v<v>>v<v<^v<>^<>^<<<>vv>vvv<>>>>vvvvv><<>>^^^v><v<v^<^><^v<vv<><^>^>^^^>v>>v^<>v<^<v^^v><<v^>>>^^^<<^vv>>^<<^^<v>^<>>>vv<<<v^v<>vv<<>v<vvvv>vv<><vvv^v^v<^v>^vv<>>vv<>><>>v<^^<<^^v>v^^>^vv>v<<vv>^>^<<<vvvv>>^^<<v^^<vv>><^<<^><v<vv<><<>>v<vvv>v><<>v>>>v>^v^>><<<<>><<<>^><^>^^<<^^<<>>^vv<><>>^<>v<<<>>v<><<>><v^<<^vv<><v^^><>>><><^^^^<>v>v<vv^<>v^v^^<v<>>v<>><<<>v>><^<>^^>^^>>^<^v^>>^^^>^v<<v<^vv>v<vv><><><>^vv<>v<<<^><v<^>v>vv^^v<>v>v^><^v<^>v>><>>^>vv<><v><vv>^>><><<>><vv<^>>^v>>><<>>v<<v<^^<v<>v><<<<>>>v>vv^v^v^vv>>>>vv>^<><>>v<>^<<^>v>^>>><v><^><><<>><>v><<>^>v<^v>>^^><v<<<<<<vvv^><>^<<<<v>vv><^>>^^vv^^v<<>^>><v^<<>v>vv^^>>^><<<v>vvv>>><v><>>vv^>>><<v>>><v^vv><^vv<>^^^^><<<<^vv<<v>^v<<><v<>^<<<vv>>^><v<^v>^v>>>v<>v><<>vv^<^^^v^v^><>v<>>vvvv<<>^^<vv>^><v<^^^^<>>^v>>vv<vv^>v
^<>>^^^<<^>^>>^v^^>>>><^<>>v<>^>v<<<vv^^<<><^><^^<vv^>^^>v<^^v<<v<v^vv<<v<v>^><^<^v^<^v<^v^<^v<<>v^>>v>vv<<^vv^v><<v<v>><><v<v<v>v>><<^<<v>><^^^^vv^vvv>><>>v^^<>v<<<v^^^^><v<<<<>^^v^^vvvv<>^^v<<v<<^^>v^^<^><^>^<^vv<^v^v^>v^><>v^v<<>^<^>v>>v>>^v^^^v><>>>>>v<<<v>>v<>v<^<>v^^v<vv<<<^<^><<>^>vv^<<>v^^vv>>^>>>vv<vvv<>>^v^^<^>>vv>v^v^><<^><v>^vvv<v>^v>^vv^v><^^^<<^<<^^^<<>^<>^v^^<^^^^>>><vv<>>vvv>v^<v>>^<><vv><^v>>>v^<<v^>>>v^>>>><^v<v^^>><v^<^vv<v><<^^v^>>>^<><<><>>^<^>^>^<^<^v^vvv>>vv^v<^v^v>^<v>>>>^v^<>v<^^^>^^^>^<^>^>^>^vv^v<vv^^v^>^>>v^>v<v^^^^v^><v><^v^v><vv>v>^vv<vvv<<>v>^vv^>v><^^>vv>v><<^>v^vvvvv><>^^v^^>v<v<^v><<v<^>^<><v>v><vvv^^<v<v<v>v^^v^<>vv<v^<v^^<^^vvv>>^^>><vvv>v^^^vv<^>v^><<v^vvv^<v>vvv<>v^<v>^vvv<>^>vv^<^v<^^><<<<<>v>^v>^<^vvv<v>^<>>>^<<<^>v>^vv^<v<v>>v^<<<^^v^<><v>^vv>^^^>vv<v>v^<<<^>vvv^v><v^<<^<>^<^>>vv>v^^<v>vv^><<v<^<^<<^<^^^<v^>^v><<^v><<^><><<<><>>v>^<><vv^^>^<<<>^^v^><v^v<<<^^>vv<<v><^<<<><v>^^v^v^<<<><^<^^>v^^><>^<^^^>^^<><>^>>vv>vv>v^^<>>v^<>^^<^^^><^vv^>><>^><v
><<^v<v<^v>^>^^v<<<^v>^<>v>^^><^<v>^^v>vv<v>^>v>><<v^^>^^<<<><>><><>v^<>^^<<>^vv>>^^^<v<<>^^^^v<^<^<^><^<^^><^>v^><^>^>>^>>^^v>><<v>^<<^v><vv>>^^v<>>vvv^<^v^^<^^<>v><<^vv<v<v<^>^v>vv><^<>v>^^>vv>v>^^^>v<<v><^v^>^>>^>v<<<vv<v<v^<^>>^<^v>>v<<<vv^v>><vv<v>^<^^>v<v>v>vvv^^<><>v<^>^v^v>vv<^<^v><<><v>>v^^^^^vv^^<^><>>v^><^>>^<<^>^^>^<^v<v^><v<<^>>vv>>v>>^v><>vv^v>v<v>>v^<>vv><>v^^^>^^<<^<v<<v<vv>^>>vv>^<^^>^<<v<><<>^<<^<<^v^>>^>v><><^vv>>vv^v>>^v<vv<^<v>v^v<vv>>v^<vv<vv>>^<v><>^v<^>><^<<v>^^^v^v>^^<^^>><^<>v^<^<v>>^vvv<v<v<^<<^<>><v>v>v<^<v^>^<>^^^v>><v<>^^v><>>^<<^><v^<<>v>^>>^>v>>^><v><><<^v<<^v<<<><<v>^v>^>>v>>v><><><<v><^v<^<v^vv>^<vv^>>v<<vv<v^^>v^^v>^v><<<^>^<><><<^<v><>v^<<<>^>^>v>v>vv<v^^<^^^v<v>^<vvv<v<>^v<^^<<<^^>v<>vv<^^^><<>^^<v<v^^^v^vvv<^v^>^><v<<<^>v>^>v>v<^vvvv>^><<vv^<<<<^v<v<vvv^><v>^<>^<<<<v<><v>vv>^vvv<<<>^<^v>vvvv<>vv<>^<<>v<<vvv<>v>^<>^v^<^<v><vv<^><^^>>vv>>v>vvvvvvv<^<v<>v>>>>>>^^^vv><v^v<<<^v^^vv<v><v<^v<<v<>>>^<^^>><<>^<><^vv><><><^<^v^<^><v^<^>><><^^>v>>^>vv^v<>><^v
<v<><<>><v>>v<<^<>><>v<>vv^^vvv<^v>>>><v^><>>vv^^<vvvv><<><v>^^<^>^><^^<^>^^v^^^^>><>v><<<<v<>vv><<vv<vv><v<v<^<^v>v<>v^>>><^<^v<><^v<v>v>>^<^>^<^^>^v<<>v>^v>vv^<v^>v^<vv^<v^v<vv^vvvv<<>><^^vvv^v^<^v<<<<v^<>^v<v>v^<v><<>v>^v<>v^v<vv>v<>v<<><<>v^^^<v<^<v^v^<v^>^>^><<><<^v>^vv<<<>vv^<<^<>^^^v^^<v^v^v<^<>^>v<><<>^v^<<^v<<^<^^vv^v>v>^><>v^>^>><v<^<vv<^<<>v^v>>>v>>^^v<^<vvv<<v>^<<v<<^^><v<v^<^<vvvv<<v>^<^^v<>vv><^^^>v^vv<>^^^<^>>v<>^v^^<v<>>v<<<^<>^v<v<^<>>v>><<^<^v^<<><><^v^^v<^^^v<v>^^^vv<<^vv<<v>>^vvv>v^>v<>^^<<<>v<v<<^vv>v>^>v^<^^^>^^^^>^>>>vv>>v<v>^<v^^<>>vv><^^>>^^>><^^v<^<^<<^<^>^v<^v<>^<>^>v<vv<v^v<<>>vv<^v<^>^><<><v^><^<>^>>>>v^^<>>>^v<v^><<><>>>>^^^>v<^v><v^^<>v>>>v<^v^<>v^><<^>vvv>>^<^<>^<^v^>^^<vv<>v>^<><<v><>^v>>>v<^v><v^>^><<<v^<<v><>^><^^><vvv>>><v^^v><<v<^>^>^><^>v<>^^>vv<^<>>><^^v^>^vv<<><v>^v>v>>>>vv>v>v<v<><>>>v^^<<>><^^><><><v<<>^><^>^vvv^><>>>v><<v<^>^vv^^v<vv>>vv>v<^^<>>^><>>><v<<<vv^vv^^^><<v>>vv^<<^<>v<^<vv>>v<>>^<>^>^<>v><^^^^vv><<^<<vv>vv^>^^>vvvv>v><>^<<>><<<>^>v^
^^^<<>v<<^^<^^>^<v>v^<^^>^v<<^>>v><^v><<<><>^v<^v>^v<>>>vvvvv>v^^<<>^>>>>>>>vv>>v><>v>vv>v<>v^vv^v^>v><<<^^<^v^v^<^<^^^^><><>>>v><><^vvv<^>vv^^<><<v^^vv^<v><^>vvv^<<<^<<<<<<>^>><>^>vv^>v^<>vv<><<^<<<>><>v>v^^v^^<>^v^<<>><^v<<>><^v>^>v^>v^^vv>>v<>^<^<^>>^>vvv>v^<vv>>v<>>v<^<v>^<^<v<v<^>><<^v<>v^>>><^vv>vv>vv>^>^<>><<v><v^>>>>v<>^<^<><^^^v><<<v><<<<>^<^v<^vvv^<^v^^>^>>v>^><v<vvvv<>^v^vv>v<<v<>>^>^^vv<<<^v<^<<<<v^v><>>><v^^>^v^^^>>vv<>^<<>v<v^>^v><<><><v<v^^<vv<<>^>>><^^v><^>^><><<v^^<vv^<<v^v<<<>^^<^>>v<<>><<<<<v<v^^^<v><^^v>>^>>>^<v><v>>vvv>^^>vv^^^^<<><<^><<<><vv<<v^>>vv>>^vv^^<^^^>>^>v>><vv^><^>^<>^><<><><vvv><^^>^^<vv>v<^>v>v>>>>^<^^^<<<v<v^v^><^^^^v^<^v<<>^v>>^<>vv>v^^<<v><>>>^^v>^<v^vv^<v<>^v<>v^><v<^v^>v^^<^vv^^^^<^<^vv^<^<^^<>^v>^<<^v>^v<<<>^v<^v><^><v^>^v<<vv^^>^<v<^^v><vv>v^vv^^v>v^>vv<<^v>v>><^v<^^<v<>^>^<<<<>^<>v<<<<^^<<^<^v>^vvv><>><vvv>^<^^v^v^><><><v<vvv^v<<v<^vv^>v<^^<<>v<<>^><><<v^<>><>>v<>^<^^<v^>^><vvv^^>^^><<<<>^^^<<v<^v><v>>^v<v>>>><^^>v<vv<<v^v^<<<<<v^>><vv^v><<^>>v
^><vv^vv^^vv>v>^v^v^^<>^v<^>v>^>vvvv>vv<v<>>><>>^^<v<<^<vv<^>v>v>^>>>^<<^><^>>>>vvv<v^>^^<^^v<>^>>^vvv^^>^v>>vvv<^^<^><>>^>>v^v><<>v>>^>^v>>^<^^vv><<^v^>>>^^^^>>vv><v<<^^<v>^^>^<<^>><>><^>>vvv<^<^<v>^<>^>vv^><<>^<<v^<<^v^v<v^^<v<^^^><^v<v>vv^^>^^vv^v<>^^v^v<>>>^vv>v>>^^v<^v^^>><vvv<>v>>^v^v>^><v<^^^^><<>^<^><>v>v>>^v^^^v^^<<v>^<<>^<><<^<^>v<<<^^^>v<>v^vv^v<^v^^<>v<^<>vv<><<>v<v^>>vv<<vv>^^^<<<v<vv^<>^v<v<vv<<<v^<^v>>^<<v><v<>^v^^>>v^<v><^v<^^><^v<>^>v^^v<vv^><><vvv<vv>v><^>^<>v^<>v<^>>><v^<^<><<^<^^>>v^^v>^^^^^<>>^v^^>v<><>vv<^<v>>v>vv^^^<<^v<>v<^^^><<v<^v><<<><<>vv<<v^^v><v<>v>^v^>>>v^<>v<vv>vvv<v><>>><<>v^<^<v<<>>><<^vv<><v>>^>^<<<<v^>>>^^>^v>>v<^<^^>^<v<v><<<^v><vv^>^>^vvvv>^>v>^v^<<>v^v<<<v>>^^>^><>^<>v^>>vv^^>^><vv>v^<^<<<^^<>>>>><^>^^><><^>^^<v<>><>>^v>><<>>>><^<<<^vv>>>^<v<^^v^v^<><>>^^>^^>><v>>>><v>>v^v^v<<v<v>>v<>^v^>>>v^<<<^<><><^<v<^>>>^>v>^vv^>v<v<v>^<>^<vv^<v^><<v^^>^^v><<v>v>v>^vv^vv><^<v^>v<^^<vv>v<vvv^>^v><^<>v><<<^^v>>^^vv<^^<v<^>^^<^vvv^<>^^vvv>vv^^v><v<<^^v<<v^<<><>>
^v>>^v><vv><>^v^^><^^v<>v^^^>^^<<>^v<^<v<^>>v^vv^<^v^<v^^<v^>^>v>>^<v<v>vv<vvv^<<>><<vv><>^<>v^vv^v^^^<v<vv^^><v>^<v^v>^<<>^^vvv^<v><^v^^<<^>^^<v>v<v<^<<>vv><^><>v>>><><<<><v^^^><v^<v<<<><^>vv<<<^v<<>><>^>>>>><v>>^<v<^<v^>>^<>>>v>><^^v<><<><vv<^<^^>><^>^vv<<<v<<^><^><<^^v^^^^<^v><v>><vv<<v>v>^^>^^>>v>^^vv>v<^^^<v^<<<^v^^>vv<^^<^>vvv>^<v^>^^vv<<v^<<>v^>vvvv>^v>><^>v<vv>^>><^^^<vv<>v^^>^^>^v<><<<^>>vvvvv<<<<v>>v^><>^^v><<v><v^v^^v><v^<<^v^<^>v<v<<<<>vv>^^<^<<^<>^><^<v<^v^>^>>v>v^>><v<><>^^vv<^><v^<<<^v^^>v<>>^>>>v<>^^v>v<<v>^^<^^v<<>vvv<v<>><^vv^v>>v^^<v>^v^vv<v^>vv^^<v<>^<><<<>>v^v>^<<vvv>^^><^>v><>v^^^^><^<>^v^^>><v^^v^^v<^<^^^v^vv>^v^vv^v^^v^^<^<^^^><^>v<vv<><>^^>v<^^^v<<<^^vvv<v>v^<^<<<v^<>>>^<>vv<>v^<>^^^v<>vv<^><v^<^>^^^<v<<>^<<v^><<<<v>v><v^>v>^v<vvvv<^<<vv<^<>^^<vvvvv^<<^<vvv><vv<^^v<<><^<>><v<>v>v><vv>>v><v>^<v>>><<<^^>><v^^^vv><^v<v><<vv^vv^vv<^>^><>>v^^<>>>>v>vv^<v<>^v>v>^>^>v<<>v^v>vv^^>v^v><<^>>v><>v>>^<v^^>v><^<<<>><v>^><>^<vv>^^<^<v>^v^v^vv<^>>v<^^^><<^><vvv><^v><<<<<>vv<<
>^^v^v^<v^<<v^^^<vv<>v^<>><v>><vvv^^vv<^<><^^vv^^^<v<>^><v<<^v<<<>>>^><<<><<vv>^<><v^^<<<^<<^v<vv<^^^>><v^^>>><^><vv>vv>>^<v^>>^>v>>>v^>v^>^<v>>><v>><v<vv^v>vvv<>>^<<<>^^^^<<<>>>>vv<v<^><<>^v<v^<>>v^v^>^<^>v>><vv^>v><v<<^v>^^v>v<v<<<^^<<>>v>^<^^<<>>^^<^<v>^>vvvv<v^>>vv^<^>^<>><<<<^v><vv<vv<v^><^<>>^>v>v>>><v<>v^<^<^>^<><^>>^^v^vv<>vv><^v<>><<><^<vvvv><^v^^v>>>v<<v<v^>^<>^<<><v<^<<<^<><<>>><^^^v^>>v>><<vv^^^<<^<>v>^^^v<<v><<<^>v^>^v<^v<<<>vv^^>>^>>vv<vv>^^<<v^><^<>v<^^v<v<v<v>>^>><<^<><>vv<v<v<>^>>>>v>v<<^^<<^vv>v<>v<><<<><^^^<<>v>>^^v<>vv^^<vv>^>^>>><>>^<<>^^>>^vv>v>^v<^<^vvv>v<<v<vv^<><<v^^>v>v^vv<<vv^<<^><v<^>^v><v><v^<>v^><<^^<v^><>v<<v<<><>v>v>>>>><>^<>v>^^><<>vv<^<<>v^>v^>>>^><><<<><>vv^v^><v>^^^v^v>>^v^>^>^<>>^v>^<><vvv><<<^v>><vv>>v<>>^v<^>>^vv><>v>><^^>v<<<v<<<vv>v>v<>^v^<<<vv^<^^v<^<vvvv<v<^<^><vv<>vv^>^v^<>>v<v>><>>><^^^><>vv<<<^^v>^>^v<vv^^v><v<<>>v>vvv<><^^>^>^<<vvvv<v><v^><>v^>v>vv<v<><vv>^<v<^v<vvvvv>^^<^^^<v>^vv^><<v>^vv>^>>^^^v<><>vvv<>>vv^<><><vv<^<>v><>v>^>^<v<<>>v>>^"""